In [2]:
from ultralytics import YOLO
import os


# === 데이터 설정 ===
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data_split/data.yaml"

# 모델 선택: s(가볍고 빠름) → 결과 좋아지면 m으로 확장
model = YOLO("yolov8s.pt")  # ← 여기만 바꾸면 됩니다

results = model.train(
    # === 기본 학습/안정성 ===
    data=DATA_YAML,      # 학습/검증 경로 정의된 data.yaml
    epochs=800,          # 총 학습 epoch (조기종료가 있으면 실제로는 더 적게 끝날 수 있음)
    patience=50,         # val metric이 개선 없을 때 조기종료 대기 epoch
    imgsz=896,           # 입력 해상도(정사각). 8GB GPU에서 1024보다 안정적
    batch=6,             # 고정 배치 크기 (AutoBatch 대신 보수적 고정값)
    device=0,            # GPU 인덱스 (여러 GPU면 "0,1" 형태 가능)
    workers=8,           # DataLoader 병렬 로드 워커 수 (디스크/CPU 상황에 맞춰)
    rect=False,          # aspect ratio 유지 비활성 → 셔플 가능, 일반화↑
    cache="disk",        # 이미지/라벨 디스크 캐시. 재현성↑ (ram은 비결정성 경고)
    amp=True,            # 자동 혼합정밀도(fp16). 최신 GPU에서 속도/메모리 이점
    deterministic=True,  # 연산 결정론 강화
    seed=0,              # 위와 세트 (결정론적 학습 재현)

    # === 온라인 증강(안전하게 약하게) ===
    mosaic=0.05,         # 초기 모자이크 비율. 소물체 리콜↑. 과하면 artifacts↑
    close_mosaic=10,     # 지정 epoch 이후 mosaic=0 자동 전환(과적합 방지)
    mixup=0.0,           # MixUp 비활성(디봇/잔디 경계 왜곡 방지)
    copy_paste=0.0,      # Copy-Paste 비활성(객체 경계/텍스처 왜곡 우려)
    hsv_h=0.01,          # Hue 변동(작게). 잔디 색 왜곡 최소화
    hsv_s=0.08,          # Saturation 변동 (0.2→0.15)
    hsv_v=0.08,          # Value(밝기) 변동 (0.2→0.15)
    degrees=0.0,         # 회전 금지(지형/그림자 방향성 유지 목적)
    translate=0.05,      # 이동 변형 (일반화 최소한 확보)
    scale=0.10,          # 스케일 변형 (소량만 허용)
    shear=0.0,           # 기울이기 금지(텍스처 왜곡 방지)
    erasing=0.0,         # 랜덤 Erasing 비활성(작은 객체 소실 방지)
    # ※ 오프라인 증강을 이미 다수 수행했다면 온라인은 약하게 유지하는 편이 안전

    # === 손실/최적화 쪽 튜닝 ===
    box=7.5,             # bbox 회귀 손실 가중(기본≈7.5). 위치 정밀도 기본값
    cls=0.5,             # 클래스 분류 손실 가중(기본 수준)
    dfl=1.5,             # Distribution Focal Loss 가중(경계 정밀도↑)
    # lr0/lrf: Cosine 스케줄(초기 lr → 최종 lr=lr0*lrf)
    lr0=0.0015,          # 초기 lr (AdamW 기준 안정값 / SGD면 보통 0.01)
    lrf=0.01,            # 최종 lr 비율 (0.01이면 마지막에 lr0의 1%)
    momentum=0.9,        # SGD: 모멘텀, Adam/AdamW: beta1 역할 (ultra 내부 호환)
    weight_decay=0.01,   # AdamW 권장: 0.01 / SGD 권장: 0.0005

    # === Optimizer 선택 ===
    # optimizer 인자를 생략하면 기본 SGD로 동작
    # optimizer="SGD",     # 대규모 데이터/보수적 일반화. 권장 세팅: lr0=0.01, wd=0.0005, momentum=0.937
    optimizer="AdamW",    # 소/중규모+증강 多에서 빠르고 안정적 수렴. 권장: lr0≈0.0015~0.002, wd≈0.01
    # optimizer="Adam",    # 빠른 수렴(초반 성능↑)이나 과적합 주의. 권장: lr0≈0.001~0.002, wd 0~0.0005
    # optimizer="RMSProp", # 특수 케이스 외 권장X. 튜닝 민감.
    # optimizer="NAdam",   # Adam+Nesterov. 드물게 사용. 실험용.

    # === 출력 경로 ===
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # 상위 폴더
    name="20250912_merge_data_yolov8s_safe",  # 실험 이름(여러 실험 구분용)
    exist_ok=True,        # 같은 이름 덮어쓰기 허용
    save_period=50,       # 50 epoch마다 중간 저장(롱런 학습 중단 대비)
    plots=True            # 학습 곡선, confusion matrix 등 자동 저장
)

    460/800      3.01G     0.3805     0.2134     0.8129          6        896: 100%|██████████| 865/865 [01:37<00:00,  8.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:04<00:00, 14.25it/s]

                   all        798       2543      0.973      0.973      0.987      0.856



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    461/800      3.01G     0.3761     0.2118     0.8108          6        896: 100%|██████████| 865/865 [01:36<00:00,  9.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:04<00:00, 14.82it/s]

                   all        798       2543      0.973      0.973      0.987      0.857


...

    593/800      3.01G     0.3175     0.1848     0.7983          8        896: 100%|██████████| 865/865 [01:38<00:00,  8.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:04<00:00, 14.42it/s]

                   all        798       2543      0.969      0.973      0.986      0.861
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 543, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



593 epochs completed in 17.079 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250912_merge_data_yolov8s_safe/weights/last.pt, 22.6MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250912_merge_data_yolov8s_safe/weights/best.pt, 22.6MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250912_merge_data_yolov8s_safe/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,128,680 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:04<00:00, 15.11it/s]


                   all        798       2543      0.972      0.971      0.986      0.863
                 Divot        508        896      0.991      0.966      0.987      0.812
           Fixed_Divot        666       1353      0.971      0.933      0.976      0.749
        Diseased_Grass         78        107      0.909      0.938      0.955      0.827
       Confused_Object          4          5      0.934          1      0.995      0.995
                  Pole         44         44          1      0.993      0.995        0.9
             Sprinkler         12         12      0.977          1      0.995      0.867
                 Drain         70         70          1      0.993      0.995      0.887
             Golf ball         56         56      0.994      0.946      0.989      0.865
Speed: 0.3ms preprocess, 3.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250912_merge_data_yolov8s_safe
